In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,2.811236e+08,245.59,-734.69,246.69,-88.07,-62.86,-1.02,3.11
3,2.455900e+02,15362629.60,-162.33,9.89,53.83,23.27,2.16,-0.05
4,-7.346900e+02,-162.33,1069895.71,73.95,3.18,-2.94,-0.63,0.19
5,2.466900e+02,9.89,73.95,486139.05,-16.51,-1.00,0.49,-0.05
6,-8.807000e+01,53.83,3.18,-16.51,133268.88,-1.79,0.21,0.02
7,-6.286000e+01,23.27,-2.94,-1.00,-1.79,21386.17,0.07,0.01
8,-1.020000e+00,2.16,-0.63,0.49,0.21,0.07,1439.97,-0.01
9,3.110000e+00,-0.05,0.19,-0.05,0.02,0.01,-0.01,63.01


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00000,-0.00004,0.00002,-0.00001,-0.00003,-0.00000,0.00002
3,0.00000,1.00000,-0.00004,0.00000,0.00004,0.00004,0.00001,-0.00000
4,-0.00004,-0.00004,1.00000,0.00010,0.00001,-0.00002,-0.00002,0.00002
5,0.00002,0.00000,0.00010,1.00000,-0.00006,-0.00001,0.00002,-0.00001
6,-0.00001,0.00004,0.00001,-0.00006,1.00000,-0.00003,0.00002,0.00001
7,-0.00003,0.00004,-0.00002,-0.00001,-0.00003,1.00000,0.00001,0.00001
8,-0.00000,0.00001,-0.00002,0.00002,0.00002,0.00001,1.00000,-0.00003
9,0.00002,-0.00000,0.00002,-0.00001,0.00001,0.00001,-0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1190.2224563750203

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[9.87941818e+15 2.89023098e+13 1.37724475e+11 2.68862012e+10
 1.89722286e+09 4.34697902e+07 1.48221331e+05 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,9.999692e-01,-0.007828,-0.000546,-0.000246,-0.000068,-0.000011,-7.361396e-07,-2.099367e-08
1,7.819984e-03,0.999886,-0.011676,-0.005276,-0.001441,-0.000230,-1.546943e-05,-6.882989e-07
2,5.990940e-04,0.010970,0.993376,-0.111856,-0.023604,-0.003723,-2.511748e-04,-1.077474e-05
3,3.484024e-04,0.006393,0.109642,0.990918,-0.076766,-0.011178,-7.499238e-04,-3.293718e-05
4,1.193259e-04,0.002175,0.031629,0.072956,0.995261,-0.055815,-3.626662e-03,-1.585505e-04
5,2.564932e-05,0.000463,0.006679,0.014713,0.054542,0.997800,-3.402535e-02,-1.478336e-03
6,2.568935e-06,0.000047,0.000672,0.001477,0.005389,0.033662,9.984573e-01,-4.379824e-02
7,2.138638e-07,0.000004,0.000059,0.000130,0.000472,0.002943,4.372178e-02,9.990393e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[3.0824154230280065e-05,
 0.00011372305906431901,
 0.006623712345669608,
 0.009081701339605508,
 0.004739359030728751,
 0.002200416820812734,
 0.0015427382497267272,
 0.0009607104841452552]